In [ ]:
#Google cloud 불러오기 (공유 폴더)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.core.splitinput import LineInfo
!pip install googletrans==3.1.0a0
from googletrans import Translator


log = open("/content/drive/MyDrive/OpenSource/kakaotalk_log.txt", 'r')
line = log.readline()
line = log.readline()
line = log.readline() #첫 세줄은 무시한다.

translator = Translator(raise_exception=True)
print()
user_name = "[박준석 형]" #input("자신의 이름을 입력하세요: ")
other_name = "[이민형]" #input("상대방의 이름을 입력하세요: ")
user_name_length = len(user_name)
other_name_length = len(other_name)
chat_log = ""

while True :
    line = log.readline() # 문장단위로 읽어옴
    if not line : 
      f = open("/content/drive/MyDrive/OpenSource/result.txt", 'w')
      f.write(chat_log)
      f.close()      
      log.close()
      print(chat_log)
      break

    if line[0:user_name_length] == user_name :
      line = log.readline()
      continue
    if line[0:2] == "송금" :
      line = log.readline()
      continue
    if line[0:2] == "받은" :
      line = log.readline()
      continue
    if line[0] == '-' :
      line = log.readline()
      continue
    
    newline = line[other_name_length + 11 :]
    newline = newline.strip() #상대방 대화로그 추출
    trans_log = translator.translate(newline).text

    chat_log = chat_log + " " + trans_log





Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.3 MB/s 
     |████████████████████████████████| 42 kB 958 kB/s 
     |████████████████████████████████| 1.4 MB 10.0 MB/s 
     |████████████████████████████████| 53 kB 2.0 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16368 sha256=08b095323af144a8a3553429c5f89b02de89c4336ce249e8ab6b9f8f2036e39d
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans

 Do you hear me too? a lot of dogs where are you haha I see.. Rice after... ? Oh no way it's over Have you heard of circuit theory? Photo come here Is this your classroom?? Elbe.. I have a class at 1:30 How old are you? Ha ha ha ha go like that me in 15 minutes possible .. Hyung, I have something to do before class. 

In [ ]:
#MBTI 분류 알고리즘(오픈소스)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.metrics import classification_report, f1_score, roc_curve
import pickle
import os.path
import plotly.offline as pyo
import plotly.graph_objs as go
import spacy
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import precision_recall_curve # PR curve

data_dir='./drive/MyDrive/OpenSource/dataset/'

train = pd.read_csv(os.path.join(data_dir,'MBTI_train.csv'), encoding='ISO 8859-1', header=None, names=['posts', 'type'])
test = pd.read_csv(os.path.join(data_dir, 'MBTI_test.csv'), encoding='ISO 8859-1', header=None, names=['posts'])

print(train.shape, test.shape) # 74357, 9337
train.head()

(41814, 2) (13725, 1)


,posts,type
0,posts,type
1,know intj tool use interaction people excuse a...,INTJ
2,rap music ehh opp yeah know valid well know fa...,INTJ
3,preferably p hd low except wew lad video p min...,INTJ
4,drink like wish could drink red wine give head...,INTJ


In [ ]:
test.head()

recreate_model = False

filename = 'mbti_svm_v2.sav'

if not os.path.isfile(filename):
    recreate_model=True
    
X=train['posts']
y=train['type']

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 재생성 여부 확인
if recreate_model:
    
    # vectorizer 정의 및 fit_transform
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(X_train)
    
    # 훈련
    clf = LinearSVC()
    clf.fit(X_train_tfidf, y_train)
    
    # vectorizer 및 모델 파이프라인
    text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
    text_clf.fit(X_train, y_train)
    
    # 모델 저장
    pickle.dump(text_clf, open(filename, 'wb'))

# 모델 재생성하지 않으면 기존 저장된 모델 불러오기
else:
    # loading the model from disk
    text_clf = pickle.load(open(filename, 'rb'))

In [ ]:
#저장된 결과를 확인
predictions = text_clf.predict(X_test)

print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

        ENFJ       0.85      0.70      0.77       174
        ENFP       0.87      0.83      0.85       738
        ENTJ       0.90      0.82      0.86       375
        ENTP       0.84      0.86      0.85      1176
        ESFJ       0.75      0.57      0.65        21
        ESFP       0.93      0.56      0.70        48
        ESTJ       0.91      0.79      0.85        52
        ESTP       0.92      0.92      0.92       232
        INFJ       0.81      0.84      0.83      1177
        INFP       0.83      0.87      0.85      1218
        INTJ       0.79      0.84      0.81      1249
        INTP       0.80      0.82      0.81      1189
        ISFJ       0.82      0.57      0.68        87
        ISFP       0.85      0.68      0.76       103
        ISTJ       0.86      0.75      0.80       136
        ISTP       0.92      0.84      0.88       388

    accuracy                           0.83      8363
   macro avg       0.85   

In [ ]:
# 평가지표 기본 자료 : Confusion_matrix
confusion_matrix = metrics.confusion_matrix(y_test, predictions)
print(confusion_matrix)

# 평가지표 : Accuracy
score = round(metrics.accuracy_score(y_test, predictions) , 10)
print(score)
# 평가지표 : F1 Score
score = round(metrics.f1_score(y_test, predictions,average='weighted'),10)
print(score)
# 평가지표 : Recall Score
score = round(metrics.recall_score(y_test, predictions,average='weighted'),10)
print(score)

[[ 122    3    3    6    2    0    4    0   17    8    5    2    1    0
     1    0]
 [   2  616    4   12    1    0    0    0   31   40   22    8    0    0
     2    0]
 [   0    2  308   10    0    0    0    0    9    5   22   17    0    1
     0    1]
 [   0   12    3 1007    0    0    0    3   26   22   39   50    1    2
     3    8]
 [   1    1    0    1   12    0    0    0    1    3    2    0    0    0
     0    0]
 [   0    2    1    6    0   27    0    1    0    4    4    1    0    1
     0    1]
 [   0    0    1    2    0    0   41    0    2    1    1    3    0    0
     1    0]
 [   0    0    2    5    0    0    0  213    2    4    1    4    0    0
     0    1]
 [   7   16    2   31    0    0    0    1  989   66   38   20    2    2
     1    2]
 [   1   30    2   14    0    0    0    1   51 1062   21   29    2    3
     1    1]
 [   3   16    6   25    1    0    0    7   36   16 1044   88    0    2
     2    3]
 [   5    2    8   49    0    0    0    3   27   25   86  973    

In [ ]:
# 평가지표 : Accuracy
print(f"Overall accuracy of the model: {round(metrics.accuracy_score(y_test, predictions),10)}")

Overall accuracy of the model: 0.8322372354


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve

def plot_roc_curve(fper, tper):
  plt.plot(fper, tper, color='red', label='ROC')
  plt.plot([0,1], [0,1], color='green', linestyle='--')
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver Operating Characteristic Curve')
  plt.legend()
  plt.show()

model = RandomForestClassifier()
model.fit(X_train, y_train)
prob = model.predict_proba(X_test)
prob = probs[:,1]
fper, tper, thresholds = roc_curve(y_test, prob)



ValueError: ignored

In [ ]:
result_dir='./drive/MyDrive/OpenSource/'

test2 = pd.read_csv(os.path.join(result_dir, 'result.txt'), encoding='ISO 8859-1', header=None, names=['posts'])
predictionss = text_clf.predict(test2['posts'])

print(predictionss)

In [ ]:
# submission
sub_dir = './sub'
if not os.path.exists(sub_dir):
  os.mkdir(sub_dir)

sub = pd.DataFrame(predictions)
assert sub.shape == test.shape
sub.to_csv(os.path.join(sub_dir, 'baseline.csv'), index=False, header=None) # no header
sub.head()

--------------------------- 평가지표를 위한 코드 -------------------------------

PR Curve : https://www.kaggle.com/code/nicholasgah/optimal-probability-thresholds-using-pr-curve 참조

3D PR Curve : https://ardentdays.tistory.com/20 참조

matplotlib : https://wikidocs.net/92071 참조

ROC Curve : https://ardentdays.tistory.com/21 참조

AUC : https://jimmy-ai.tistory.com/345 참조

--------------------------------------------------------------------------------

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

y_true = np.array([1,1,0,1,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,0])
y_scores = np.array([0.9,0.8,0.7,0.6,0.55,0.54,0.53,0.52,0.51,0.505,0.4,0.39,0.38,0.37,0.36,0.35,0.34,0.33,0.3,0.1 ])

def get_recall(y_true,y_scores,threshold):
    predict_positive_num = len(y_scores[y_scores >= threshold])
    tp = len( [x for x in y_true[:predict_positive_num] if x == 1] )
    ground_truth  = len(y_true[y_true==1])
    recall = tp / ground_truth
    return recall

def get_precision(y_true,y_scores,threshold):
    predict_positive_num = len(y_scores[y_scores >= threshold])
    tp = len( [x for x in y_true[:predict_positive_num] if x == 1] )
    fp = len( [x for x in y_true[:predict_positive_num] if x == 0] )
    precision = tp / (tp + fp) 
    return precision 

def recall_precision_plot(y_true, y_scores):
    recall, precision = [] , []

    for _ in y_scores: # y_scores 를 thresholds 처럼 사용했음
        recall.append(get_recall(y_true, y_scores, _ ))
        precision.append(get_precision(y_true,y_scores,_))
    
    fig = plt.figure(figsize = (9,6))
    plt.plot(recall, precision)
    plt.scatter(recall,precision)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve 2D')
    plt.show()
    
recall_precision_plot(y_true,y_scores)

In [ ]:

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from scipy import interp
from sklearn.metrics import roc_auc_score

# Import some data to play with
iris = filename.load_iris()
X = iris.data
y = iris.target

# Add noisy features to make the problem harder
random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
X = np.c_[X, random_state.randn(n_samples, 200 * n_features)]

classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True,
                                 random_state=random_state))
y = label_binarize(y, classes=[0, 1, 2])
n_classes = y.shape[1]
y_score = classifier.fit(X_train, y_train).decision_function(X_test)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])



# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
#특정 클래스에 대한 ROC 곡선 플롯

 

plt.figure()
lw = 2
plt.plot(fpr[2], tpr[2], color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % roc_auc[2])
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")
plt.show()